In [ ]:
%load_ext autoreload

In [ ]:
import altair as alt
import polars as pl

%autoreload
from altair_utils import color_value_transit_teal

In [ ]:
baybridge_monthly = pl.read_csv(
    r"Q:\Data\Observed\Streets\BATA\BayBridge-westbound-toll_crossings.csv"
).with_columns(date=pl.date(pl.col("year"), pl.col("month"), 1))
baybridge_quarterly = (
    baybridge_monthly.filter(
        (pl.col("year") > 2018) & (pl.col("date") <= pl.date(2025, 1, 1))
    )
    .with_columns(quarter=pl.col("date").dt.quarter())
    .group_by("year", "quarter")
    .agg(pl.min("date"), pl.sum("Bay Bridge (westbound) toll crossings"))
    .sort("date")
)
baybridge_annual = (
    baybridge_monthly.filter(pl.col("year") < 2025)
    .group_by("year")
    .agg(pl.sum("Bay Bridge (westbound) toll crossings"), pl.min("date"))
    .vstack(
        # from Q:\Data\Observed\Streets\BATA\Through May-2023\San Francisco-Oakland 1-1-15 through 05-2023.xls
        # spot checked other years, numbers are basically the same (with a <50 difference)
        pl.DataFrame(
            {
                "year": [2018],
                "Bay Bridge (westbound) toll crossings": [48119826],
            }
        ).with_columns(date=pl.date(pl.col("year"), 1, 1))
    )
    .sort("date")
)

In [ ]:
baybridge_quarterly_chart = (
    alt.Chart(baybridge_quarterly)
    .mark_line()
    .encode(
        x=alt.X("date:T"),
        y=alt.Y(
            "Bay Bridge (westbound) toll crossings:Q",
            # title="Golden Gate Bridge (southbound) annual vehicle crossings",
        ),
        color=color_value_transit_teal,
        tooltip=["year", "quarter", "Bay Bridge (westbound) toll crossings"],
    )
).interactive()
baybridge_quarterly_chart

In [ ]:
baybridge_annual.drop("date").write_csv("output/data/screenline-baybridge.csv")

baybridge_chart = (
    alt.Chart(baybridge_annual)
    .mark_line()
    .encode(
        x=alt.X("date:T", title="year"),
        y=alt.Y(
            "Bay Bridge (westbound) toll crossings:Q",
            # title="Golden Gate Bridge (southbound) annual vehicle crossings",
        ),
        color=color_value_transit_teal,
        tooltip=[
            "year",
            alt.Tooltip(
                "Bay Bridge (westbound) toll crossings", title="toll crossings"
            ),
        ],
    )
).interactive()
baybridge_chart.save("output/Links/screenline-baybridge.png")
baybridge_chart.properties(width="container").save(
    "output/Links/screenline-baybridge.html"
)
baybridge_chart

In [ ]:
# monthly GG Bridge crossings don't go far back enough
ggbridge = (
    pl.read_csv(r"Q:\Data\Observed\Streets\GoldenGateBridge\crossings-annual.csv")
    .with_columns(
        pl.col("fiscal year (Jul-Jun)").str.strip_prefix("20").str.replace("/20", "/")
    )
    .filter(pl.col("fiscal year (Jul-Jun)") > "16/17")
)
ggbridge.write_csv("output/data/screenline-ggbridge.csv")

ggbridge_chart = (
    alt.Chart(ggbridge)
    .mark_line()
    .encode(
        x=alt.X("fiscal year (Jul-Jun):O", axis=alt.Axis(labelOverlap=True)),
        y=alt.Y(
            "annual southbound vehicle crossings",
            title="Golden Gate Bridge (southbound) annual vehicle crossings",
        ),
        color=color_value_transit_teal,
        tooltip=["fiscal year (Jul-Jun)", "annual southbound vehicle crossings"],
    )
).interactive()
ggbridge_chart.save("output/Links/screenline-ggbridge.png")
ggbridge_chart.properties(width="container").save(
    "output/Links/screenline-ggbridge.html"
)
ggbridge_chart

In [ ]:
countyline = (
    pl.read_csv(
        r"Q:\CMP\LOS Monitoring 2023\PeMS\cmp_pems_volumes_all-v3-countyline.csv"
    )
    .group_by("year")
    .agg(pl.sum("volume"))
    .sort("year")
    .filter(pl.col("year") > 2017)
)
countyline.write_csv("output/data/screenline-countyline.csv")

countyline_chart = (
    alt.Chart(countyline.with_columns(date=pl.date(pl.col("year"), 1, 1)))
    .mark_line()
    .encode(
        x=alt.X("date:T", title="year"),
        y=alt.Y(
            "volume",
            title="San Mateo countyline (US-101 & I-280) weekday peaks volume",
        ),
        color=color_value_transit_teal,
        tooltip=["year", "volume"],
    )
).interactive()
countyline_chart.save("output/Links/screenline-countyline.png")
countyline_chart.properties(width="container").save(
    "output/Links/screenline-countyline.html"
)
countyline_chart